# Iris con clasificadores discriminativos

Al igual que árboles y clasificadores generativos, es muy fácil aprender y evaluar clasificadores discriminativos con sklearn.

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

Lectura del corpus iris:

In [2]:
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.4, shuffle=True, random_state=23)

El algoritmo Perceptrón es una técnica pionera del ML, ampliamente conocida. En la actualidad se tiende a ver como un caso particular de descenso por gradiente estocástico para aprendizaje de clasificadores lineales. Éste es, precisamente, el punto de vista en sklearn. Destacaremos aquí que el algoritmo baraja los datos tras cada iteración a no ser que se indique lo contrario (shuffle=False); también que puede añadirse una penalización (penalty={'l2','l1','elasticnet'}, default=None) cuya fuerza se regula mediante un parámetro alpha (0.0001 por omisión).

In [9]:
clf = Perceptron(random_state=0, alpha=0.0001, penalty='l2').fit(X_train, y_train)
acc = accuracy_score(y_test, clf.predict(X_test))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de Perceptron(penalty='l2') es 36.7%


**Ejercicio:** Estudia el efecto de alpha sobre la precisión de Perceptrón con penalización 'l2'.

In [4]:
for alpha in np.logspace(-9,0,10):
    clf = Perceptron(random_state=0, alpha=alpha, penalty='l2').fit(X_train, y_train)
    acc = accuracy_score(y_test, clf.predict(X_test))
    print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de Perceptron(alpha=1e-09, penalty='l2') es 76.7%
La precisión de Perceptron(alpha=1e-08, penalty='l2') es 76.7%
La precisión de Perceptron(alpha=1e-07, penalty='l2') es 76.7%
La precisión de Perceptron(alpha=1e-06, penalty='l2') es 76.7%
La precisión de Perceptron(alpha=1e-05, penalty='l2') es 95.0%
La precisión de Perceptron(penalty='l2') es 36.7%
La precisión de Perceptron(alpha=0.001, penalty='l2') es 65.0%
La precisión de Perceptron(alpha=0.01, penalty='l2') es 70.0%
La precisión de Perceptron(alpha=0.1, penalty='l2') es 30.0%
La precisión de Perceptron(alpha=1.0, penalty='l2') es 30.0%


Regresión logística es un modelo de clasificación lineal muy importante pues puede verse como el primer paso en la construcción de redes neuronales multi-capa para clasificación. Además, si nuestro objetivo es entrenar un clasificador lineal tan preciso como sea posible, regresión logística es el primer camino a explorar. La función LogisticRegression de sklearn incluye diferentes solvers (algoritmos de optimización) con recomendaciones sobre cómo escoger el más adecuado para la tarea concreta que nos ocupa. Por ejemplo, para conjuntos de datos pequeños se recomienda usar 'liblinear'. Todos los solvers pueden aplicarse con regularización L2 (opción por omisión) o sin ella (penalty='None').

In [10]:
clf = LogisticRegression(solver='liblinear').fit(X_train, y_train)
acc = accuracy_score(y_test, clf.predict(X_test))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de LogisticRegression(solver='liblinear') es 96.7%


**Ejercicio:** Estudia el efecto del solver escogido (con max_iter=10000) sobre la precisión de regresión logística.

In [15]:
for s in ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']:
    print(f'-------- {s} --------')
    for m in [100, 1000, 10000]:
        clf = LogisticRegression(solver=s, max_iter=m).fit(X_train, y_train)
        acc = accuracy_score(y_test, clf.predict(X_test))
        print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))
    print()

-------- lbfgs --------
La precisión de LogisticRegression() es 95.0%
La precisión de LogisticRegression(max_iter=1000) es 95.0%
La precisión de LogisticRegression(max_iter=10000) es 95.0%

-------- liblinear --------
La precisión de LogisticRegression(solver='liblinear') es 96.7%
La precisión de LogisticRegression(max_iter=1000, solver='liblinear') es 96.7%
La precisión de LogisticRegression(max_iter=10000, solver='liblinear') es 96.7%

-------- newton-cg --------
La precisión de LogisticRegression(solver='newton-cg') es 95.0%
La precisión de LogisticRegression(max_iter=1000, solver='newton-cg') es 95.0%
La precisión de LogisticRegression(max_iter=10000, solver='newton-cg') es 95.0%

-------- newton-cholesky --------
La precisión de LogisticRegression(solver='newton-cholesky') es 95.0%
La precisión de LogisticRegression(max_iter=1000, solver='newton-cholesky') es 95.0%
La precisión de LogisticRegression(max_iter=10000, solver='newton-cholesky') es 95.0%

-------- sag --------
La preci

c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [17]:
for C in [100, 10, 1, 0.1, 1e-2, 1e-3]:
    clf = LogisticRegression(solver='saga', C=C).fit(X_train, y_train)
    acc = accuracy_score(y_test, clf.predict(X_test))
    print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))


La precisión de LogisticRegression(C=100, solver='saga') es 98.3%
La precisión de LogisticRegression(C=10, solver='saga') es 98.3%
La precisión de LogisticRegression(C=1, solver='saga') es 98.3%
La precisión de LogisticRegression(C=0.1, solver='saga') es 100.0%
La precisión de LogisticRegression(C=0.01, solver='saga') es 95.0%
La precisión de LogisticRegression(C=0.001, solver='saga') es 73.3%


c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the 

Al igual que los clasificadores discriminativos en general, regresión logística es flexible en preproceso de características. Por ejemplo, aparte de las $4$ características originales, $5$ en notación homogénea, podemos añadir $4*5/2=10$ características cuadráticas y obtener $D=10+5=15$ características en total. Una manera sencilla de implementar este preproceso en sklearn consiste en crear un pipeline que preceda el clasificador con los pasos de preproceso que consideremos oportunos.

In [19]:
# print(y_train.shape)
clf = make_pipeline(PolynomialFeatures(degree=2),
    LogisticRegression(max_iter=10000)).fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('logisticregression', LogisticRegression(max_iter=10000))]) es 98.3%
